In [1]:
pip install librosa soundfile numpy sklearn pyaudio

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])

        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))

        if chroma:
            stft = np.abs(librosa.stft(X))
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))

        # Define `mel` before using it in the condition
        if mel:
            mel_spec = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel_spec))

        return result


In [5]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [14]:
def load_data(test_size=0.25):
    X= []
    Y= []
    data_path = "/content/speech-emotion-data"  # Update with the correct extraction path

    for subdir, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(".wav"):  # Ensure only audio files are processed
                file_path = os.path.join(subdir, file)

                # Pass default values for mfcc, chroma, and mel
                feature = extract_feature(file_path, mfcc=True, chroma=True, mel=True)

                X.append(feature)
                label = int(file[7:8]) - 1  # Assuming labels are encoded in filenames
                Y.append(label)

    if len(X) == 0:
        raise ValueError("No audio files found! Check the extraction path.")

    from sklearn.model_selection import train_test_split
    return train_test_split(np.array(X), np.array(Y), test_size=test_size, random_state=42)


In [15]:
import os
import zipfile

# Define the path where the ZIP file is uploaded
zip_path = "/content/speech-emotion-recognition-ravdess-data.zip"
extract_path = "/content/speech-emotion-data"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed! Files are in:", extract_path)


Extraction completed! Files are in: /content/speech-emotion-data


In [16]:
x_train, x_test, y_train, y_test = load_data(test_size=0.25)
print("Data successfully split! Training samples:", len(x_train), "Testing samples:", len(x_test))


Data successfully split! Training samples: 1080 Testing samples: 360


In [17]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1080, 360)


In [18]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [19]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [20]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [21]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [22]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 56.11%
